Part A: Linear Classification Models

Part B: Neural Network Implementation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self,num_classes=10):
        super(SimpleCNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, num_classes)
        # --- Weight Initialization ---
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.fc3.weight)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x= F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [2]:
import torchvision
import torchvision.transforms as transforms

# Define the transform: Convert images to PyTorch Tensors
transform = transforms.ToTensor()
# Load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

In [3]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64
train=DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test=DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch.optim as optim


model = SimpleCNN()

loss=nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [5]:
NUM_EPOCHS = 5
for epoch in range(NUM_EPOCHS):
    model.train()
    for images,labels in train:
        optimizer.zero_grad()
        outputs=model(images)
        l=loss(outputs,labels)
        l.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {l.item():.4f}")

Epoch [1/5], Loss: 2.4634
Epoch [1/5], Loss: 2.3847
Epoch [1/5], Loss: 2.3091
Epoch [1/5], Loss: 2.3022
Epoch [1/5], Loss: 2.2567
Epoch [1/5], Loss: 2.1764
Epoch [1/5], Loss: 2.1738
Epoch [1/5], Loss: 2.2046
Epoch [1/5], Loss: 2.0433
Epoch [1/5], Loss: 2.0164
Epoch [1/5], Loss: 1.9817
Epoch [1/5], Loss: 1.9429
Epoch [1/5], Loss: 1.8652
Epoch [1/5], Loss: 1.8835
Epoch [1/5], Loss: 1.7551
Epoch [1/5], Loss: 1.7319
Epoch [1/5], Loss: 1.7040
Epoch [1/5], Loss: 1.6856
Epoch [1/5], Loss: 1.7147
Epoch [1/5], Loss: 1.5173
Epoch [1/5], Loss: 1.5684
Epoch [1/5], Loss: 1.3104
Epoch [1/5], Loss: 1.4308
Epoch [1/5], Loss: 1.3912
Epoch [1/5], Loss: 1.1660
Epoch [1/5], Loss: 1.2248
Epoch [1/5], Loss: 1.0735
Epoch [1/5], Loss: 1.2044
Epoch [1/5], Loss: 1.1254
Epoch [1/5], Loss: 1.0830
Epoch [1/5], Loss: 1.0287
Epoch [1/5], Loss: 0.9987
Epoch [1/5], Loss: 0.8614
Epoch [1/5], Loss: 0.8429
Epoch [1/5], Loss: 0.9271
Epoch [1/5], Loss: 0.8017
Epoch [1/5], Loss: 0.9570
Epoch [1/5], Loss: 0.8591
Epoch [1/5],

In [6]:
model.eval() 
correct = 0
total = 0


with torch.no_grad(): 
    for images, labels in test: 
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        for i in range(len(labels)):
            if predicted[i] == labels[i]:
                correct += 1

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 97.29%


Part C: Comprehensive Analysis

Part D: Advanced Techniques